In [1]:
%load_ext autoreload
%autoreload 2
from characterHelpers import extractCharacter,isCharacter

In [2]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    whole_text=""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        # Get the number of pages in the PDF
        num_pages = len(pdf_reader.pages)

        # Iterate through each page and extract text
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            whole_text+=text
            
            # You can process the extracted text as needed (print, store, etc.)
            # print(f"Page {page_num + 1}:\n{text}\n")

        return whole_text

# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_file_path = 'pacific-rim-2013_1.pdf'
whole_text=extract_text_from_pdf(pdf_file_path)


FileNotFoundError: [Errno 2] No such file or directory: 'pacific-rim-2013_1.pdf'

In [ ]:
whole_text

'PACIFIC RIM\nby\nTravis BeachamPREFACE: LEXICON\n(A supplementary guide to pertinent terms and jargon...)\n•Anteverse. the older, parent universe on the other side of the \nInterstice\n•bruiser. (slang) Jaeger gunner\n•BTB. brain-to-brain interface (see P ONS or DRIFT)\n•conn-pod. the Jaeger\'s detachable cockpit module; sometimes called a Conn\n•COSDEC. Pacific Rim Combined Special Defense Corp\n•drift. (slang) a joint vision shared by Jaeger crew members while directly linked via the Pons; triggered by subconscious \nstimuli, a generally involuntary but seldom debilitating phenomenon \n•drivesuit. the Jaeger/pilot interface suit designed to monitor vital signs and to translate nerve signals to piloting input\n•ghost-drift. (slang) a kind of quasi-telepathic vision shared \n(by parties with a pre-existing Pons connection) without the \naid of a hard neural interface; commonly triggered by stress, physical proximity, or coinciding REM sleep patterns\n•headspace. (slang) virtual space 

In [3]:
import json
import re

HEADING_ENUM = ["EXT./INT.", "EXT./INT.", "INT./EXT.", "EXT/INT", "INT/EXT", "INT.", "EXT.", "INT --", "EXT --"]

def is_heading(line):
    for heading in HEADING_ENUM:
        if not line.endswith(heading) and heading in line:
            return True
    return False

def extract_time_and_scene_data(text):
    time_vocab = "|".join([
        "NIGHT", "AFTERNOON", "MORNING", "DAYS", "DAY", "NIGHT", "DAYS", "DAY", "ANOTHER DAY", "LATER", "NIGHT",
        "SAME", "CONTINUOUS", "MOMENTS LATER", "LATER", "SUNSET",
    ])
    regex = '[-,]?[ ]?(DAWN|DUSK|((LATE|EARLY) )?' + time_vocab + ')|\d{4}'
    find_time = re.search(regex, text)

    txt = list(filter(lambda x: len(x) > 0, [x.strip("-,. ") for x in text[find_time.end():].split()])) if find_time else None
    scene_data = {
        "text": " ".join(txt) if txt else "",
    }
    return txt, scene_data

def extract_heading(text):
    region_match = re.search('((?:.* )?(?:EXT[\\.]?\\/INT[\\.]?|INT[\\.]?\\/EXT[\\.]?|INT(?:\\.| --)|EXT(?:\\.| --)))', text)
    region = region_match.groups()[0] if region_match else ""
    time, scene_data = extract_time_and_scene_data(text)

    location = text.replace(region, "")
    if time and len(time) > 0:
        location = location[:location.index(time[0])]

    if len(region) > 0 and region[0].isdigit():
        region = region.lstrip('0123456789.- ')
        location = location.rstrip('0123456789.- ') if location else location
    time = time[:-1] if time and time[-1].isdigit() else time

    scene_info = {
        "region": region.strip(),
        "location": location.strip("-,. "),
        "text": " ".join(time) if time else "",
    }

    return scene_data, scene_info

def process_script(script_text):
    script_lines = script_text.split('\n')
    result = []
    current_data = {"text": ""}
    scene_counter = 1  # Initialize the scene counter
    for line in script_lines:
        if is_heading(line):
            if current_data["text"]:
                scene_detail, scene_info = extract_heading(current_data["text"])
                result.append({scene_counter: {'scene_info': scene_info, 'scene_detail': scene_detail}})
                current_data = {"text": ""}
                scene_counter += 1  # Increment the scene counter
            current_data["text"] += line
        else:
            current_data["text"] += " " + line

    # Handle the last block of text
    if current_data["text"]:
        scene_detail, scene_info = extract_heading(current_data["text"])
        result.append({scene_counter: {'scene_info': scene_info, 'scene_detail': scene_detail}})

    return result

# Example usage:
script_text = """
EXT. Forest - DAY
Some action in the forest during the day.
EXT. City Street - NIGHT
A different scene in the city at night.
"""

result = process_script(script_text)

# Save the result to a JSON file
with open("output.json", "w") as json_file:
    json.dump(result, json_file, indent=2)

print("Result saved to output.json")


Result saved to output.json
Introduction: 
Text: EXT. Forest - DAY
Some action in the forest during the day.
EXT. City Street - NIGHT
A different scene in the city at night.



In [79]:
result

[{1: {'scene_info': {'region': '',
    'location': "PACIFIC RIM by Travis BeachamPREFACE: LEXICON (A supplementary guide to pertinent terms and jargon...) •Anteverse. the older, parent universe on the other side of the  Interstice •bruiser. (slang) Jaeger gunner •BTB. brain-to-brain interface (see P ONS or DRIFT) •conn-pod. the Jaeger's detachable cockpit module; sometimes called a Conn •COSDEC. Pacific Rim Combined Special Defense Corp •drift. (slang) a joint vision shared by Jaeger crew members while directly linked via the Pons; triggered by subconscious  stimuli, a generally involuntary but seldom debilitating phenomenon  •drivesuit. the Jaeger/pilot interface suit designed to monitor vital signs and to translate nerve signals to piloting input •ghost-drift. (slang) a kind of quasi-telepathic vision shared  (by parties with a pre-existing Pons connection) without the  aid of a hard neural interface; commonly triggered by stress, physical proximity, or coinciding REM sleep patterns 

In [90]:
for index, data in enumerate(result, start=1):
    scene_text=data[index]['scene_detail']['text']

    if len(scene_text)>0:
        break
    


In [66]:
x_text=result[2][3]['scene_detail']['text']

In [67]:
def getParenthetical(text):
    """splits dialogue string into list, seperating any containing parenthetical(s)"""

    return list(
        filter(lambda x: len(x.strip()) > 0, re.split(r'(\([^)]+\))', text))
    )

In [76]:
isCharacter({"text":"RALEIGH"})

True

In [77]:
scene_text = """
The long BANSHEE WAIL of a siren stills the carefree air.
DINERS and PEDESTRIANS begin to scatter in all directions.
Flick and Newt stand frozen in their tracks.
NEWT (CONT’D)
The kaiju siren.
FLICK
A drill -- It can't be an
NEWT
No. It's real. Flick, listen.
NEWT (CONT’D)
It's okay. Listen. I've got a
scooter. We're getting out of here.
The traffic outside the cafe locks up. Impatient DRIVERS
leave their cars and run, clotting the thoroughfare with
abandoned autos. Newt leads Flick to his Vespa.
"""

def extract_scene_details(scene_text):
    # Define regular expressions for action lines, character names, and dialogues
    action_pattern = re.compile(r'^(?P<action_line>.+?)\.$')
    character_pattern = re.compile(r'^(?P<character>[A-Z][A-Za-z\s]+):$')
    dialogue_pattern = re.compile(r'^(?P<dialogue>[A-Z][A-Za-z\s]+):$')

    # Initialize variables to store extracted information
    scene_details = {"action_lines": [], "characters": [], "dialogues": []}

    # Split the raw text into lines
    lines = scene_text.strip().split('\n')

    # Iterate through each line to extract information
    for line in lines:
        # Check if the line matches the action pattern
        action_match = action_pattern.match(line)
        if action_match:
            scene_details["action_lines"].append(action_match.group('action_line'))
            continue

        # Check if the line matches the character pattern
        character_match = character_pattern.match(line)
        if character_match:
            scene_details["characters"].append(character_match.group('character'))
            continue

        # Check if the line matches the dialogue pattern
        dialogue_match = dialogue_pattern.match(line)
        if dialogue_match:
            scene_details["dialogues"].append(dialogue_match.group('dialogue'))
            continue

    return scene_details

# Extracted scene details
extracted_details = extract_scene_details(scene_text)

# Display the extracted details
print("Action Lines:", extracted_details["action_lines"])
print("Characters:", extracted_details["characters"])
print("Dialogues:", extracted_details["dialogues"])


Action Lines: ['The long BANSHEE WAIL of a siren stills the carefree air', 'DINERS and PEDESTRIANS begin to scatter in all directions', 'Flick and Newt stand frozen in their tracks', 'The kaiju siren', "No. It's real. Flick, listen", "scooter. We're getting out of here", 'abandoned autos. Newt leads Flick to his Vespa']
Characters: []
Dialogues: []


In [78]:
scene_text = """
The front of the hotel overlooks the Pacific from the top of
the seaside bluffs. Newt follows Flick out through the
outdoor cafe that spills onto the sidewalk.
FLICK
I absolve you of accountability.
NEWT
Thanks. I'm still tagging along.
FLICK
Newt. I know you're worried about
me. And that's sweet, but --
NEWT
Look, I'm going to see my old
friend, Czerny. If that's where
you're headed, you're going to have
a hard time avoiding me.
The long BANSHEE WAIL of a siren stills the carefree air.
DINERS and PEDESTRIANS begin to scatter in all directions.
Flick and Newt stand frozen in their tracks.
"""

def extract_scene_details(scene_text):
    # Define regular expressions for action lines, character names, and dialogues
    action_pattern = re.compile(r'^(?P<action_line>.+?)\.$')
    character_pattern = re.compile(r'^(?P<character>[A-Z][A-Za-z\s]+):$')
    dialogue_pattern = re.compile(r'^(?P<dialogue>[A-Z][A-Za-z\s]+):$')

    # Initialize variables to store extracted information
    scene_details = {"action_lines": [], "characters": [], "dialogues": []}

    # Split the raw text into lines
    lines = scene_text.strip().split('\n')

    # Variables to keep track of line numbers for action lines
    current_line_number = 0
    action_line_numbers = {}

    # Iterate through each line to extract information
    for line in lines:
        current_line_number += 1

        # Check if the line matches the action pattern
        action_match = action_pattern.match(line)
        if action_match:
            action_line_numbers[current_line_number] = action_match.group('action_line')
            scene_details["action_lines"].append(action_match.group('action_line'))
            continue

        # Check if the line matches the character pattern
        character_match = character_pattern.match(line)
        if character_match:
            scene_details["characters"].append(character_match.group('character'))
            continue

        # Check if the line matches the dialogue pattern
        dialogue_match = dialogue_pattern.match(line)
        if dialogue_match:
            scene_details["dialogues"].append({dialogue_match.group('dialogue'): None})
            continue

    # Assign dialogue text to the corresponding character
    current_character = None
    for i, line in enumerate(lines):
        if line.startswith(" ") and current_character is not None:
            # Append dialogue text to the current character
            scene_details["dialogues"][i - 1][current_character] = line.strip()
        elif line.strip() in scene_details["characters"]:
            # Update current character for the next set of dialogues
            current_character = line.strip()

    return scene_details, action_line_numbers

# Extracted scene details and action line numbers
extracted_details, action_line_numbers = extract_scene_details(scene_text)

# Print the extracted details
print("Action Lines:", extracted_details["action_lines"])
print("Characters:", extracted_details["characters"])
print("Dialogues:", extracted_details["dialogues"])
print("Action Line Numbers:", action_line_numbers)


Action Lines: ['outdoor cafe that spills onto the sidewalk', 'I absolve you of accountability', "Thanks. I'm still tagging along", 'a hard time avoiding me', 'The long BANSHEE WAIL of a siren stills the carefree air', 'DINERS and PEDESTRIANS begin to scatter in all directions', 'Flick and Newt stand frozen in their tracks']
Characters: []
Dialogues: []
Action Line Numbers: {3: 'outdoor cafe that spills onto the sidewalk', 5: 'I absolve you of accountability', 7: "Thanks. I'm still tagging along", 15: 'a hard time avoiding me', 16: 'The long BANSHEE WAIL of a siren stills the carefree air', 17: 'DINERS and PEDESTRIANS begin to scatter in all directions', 18: 'Flick and Newt stand frozen in their tracks'}
